# Emotion Chat Bot

In [1]:
from operator import itemgetter
from pprint import pprint

import torch
from libs import (
    EmotionModel,
    ResponseGeneratorPipeline,
    SimilarityAnalyser,
    generate_dummy_representation,
    get_sentiment_composition,
)
from libs.FullModel import (
    ChatMessage,
    create_candidates_buffer,
    get_possible_response_emotion_representation,
    get_top_emotion,
)
from sympy.core.random import randint
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
    pipeline,
)
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Load Each Module

### Response Generator

In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="hermeschen1116/response_generator_for_emotion_chat_bot",
    attn_implementation="flash_attention_2",
    pretraining_tp=1,
    load_in_4bit=True,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '


In [3]:
response_generator = ResponseGeneratorPipeline(
    model,
    tokenizer,
    framework="pt",
    task="conversation-generation",
    num_workers=16,
    torch_dtype="auto",
    add_special_tokens=True,
    truncation=False,
    padding=True,
)

In [4]:
FastLanguageModel.for_inference(response_generator.model)
response_generator.model = torch.compile(
    response_generator.model, mode="reduce-overhead"
)

### Sentiment Analyzer

In [5]:
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [6]:
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [7]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=sentiment_analysis_model,
    tokenizer=sentiment_analysis_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [8]:
sentiment_analyzer.model = torch.compile(
    sentiment_analyzer.model, mode="reduce-overhead"
)

In [9]:
print(sentiment_analyzer.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Predictor

In [10]:
emotion_predictor_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [11]:
emotion_predictor_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [12]:
emotion_predictor = pipeline(
    "sentiment-analysis",
    model=emotion_predictor_model,
    tokenizer=emotion_predictor_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [13]:
emotion_predictor.model = torch.compile(emotion_predictor.model, mode="reduce-overhead")

In [14]:
print(emotion_predictor.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Model

In [15]:
emotion_model = EmotionModel.from_pretrained(
    "hermeschen1116/emotion_model_for_emotion_chat_bot"
)

In [16]:
emotion_model = torch.compile(emotion_model, mode="reduce-overhead")

In [17]:
print(emotion_model)

OptimizedModule(
  (_orig_mod): EmotionModel(
    (_EmotionModel__weight_Q): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__weight_K): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__dropout): Dropout(p=0.5, inplace=False)
    (_EmotionModel__weight_D): Linear(in_features=7, out_features=7, bias=True)
  )
)


### Similarity Analyzer

In [18]:
threshold: float = 0.5

In [19]:
similarity_analyzer = SimilarityAnalyser(threshold)

## Combine All Modules

### Initialize

In [20]:
streamer = TextStreamer(
    tokenizer, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [21]:
system_prompt: str = ""  # input("Enter your system prompt: ").strip()

In [22]:
ideal_bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
ideal_bot_emotion_representation

tensor([0.4817, 0.4091, 0.6996, 0.5359, 0.8497, 0.1423, 0.9519])

In [23]:
bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
bot_emotion_representation

tensor([0.3156, 0.2505, 0.9316, 0.6738, 0.3336, 0.1432, 0.4173])

In [24]:
chat_buffer: list = ChatMessage(system_prompt="")
chat_buffer.show_messages()

[{content: {dialog: , emotion: }, role: system}]


In [27]:
input_text: str = "Hello, how are you?"  # input("Talk to bot...").strip()
input_text_emotion: list = sentiment_analyzer(
    chat_buffer.get_message(-1)["content"]["dialog"]
)
chat_buffer.append_message(get_top_emotion(input_text_emotion), input_text)
chat_buffer.show_messages()

skipping cudagraphs due to mutated inputs (1 instances). Found from : 
   File "/home/hermeschen/Repo/chat-bot/.venv/lib/python3.12/site-packages/transformers/models/roberta/modeling_roberta.py", line 121, in torch_dynamo_resume_in_forward_at_120
    embeddings += position_embeddings



[{content: {dialog: , emotion: }, role: system}, {content: {dialog: neutral, e ↪

↪ motion: Hello, how are you?}, role: bot}, {content: {dialog: neutral, emotio ↪

↪ n: Hello, how are you?}, role: user}]


In [28]:
input_text_emotion_composition: Tensor = get_sentiment_composition(input_text_emotion)
input_text_emotion_composition

tensor([0.3049, 0.1157, 0.1156, 0.1153, 0.1175, 0.1157, 0.1153])

### Update Current Bot's Emotion Representation

In [29]:
bot_emotion_representation = emotion_model.forward(
    input_text_emotion_composition, bot_emotion_representation
)
bot_emotion_representation

skipping cudagraphs due to skipping cudagraphs due to cpu device (primals_5). Found from : 
   File "/home/hermeschen/Repo/chat-bot/src/models/libs/EmotionModel.py", line 40, in forward
    decomposed_representation: Tensor = representation.diag().to(



tensor([0.4886, 0.2977, 0.2986, 0.2973, 0.2995, 0.2978, 0.2981],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>)

### Generate All Possible Responses In 7 Different Emotions

In [30]:
candidates_buffer: list = create_candidates_buffer(chat_buffer)
candidates_buffer

TypeError: unsupported operand type(s) for +: 'ChatMessage' and 'list'

In [29]:
generated_candidates: list = [
    result[0]
    for result in response_generator(candidates_buffer, streamer=streamer)
    if result[0][-1]["content"]["dialog"] != ""
]
print()
pprint(generated_candidates)

   neutral   Hello, how are you?     neutral    Fine, thank you. 
   neutral   Hello, how are you?     anger   
   neutral   Hello, how are you?     disgust   
   neutral   Hello, how are you?     fear    Hi! I'm terribly nervous. 
   neutral   Hello, how are you?     happiness     Fine, thank you. 
   neutral   Hello, how are you?     sadness   
   neutral   Hello, how are you?     surprise     Hi! I haven't seen you for ages! 

[[{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'content': {'dialog': 'Hello, how are you?', 'emotion': 'neutral'},
   'role': 'user'},
  {'content': {'dialog': 'Fine, thank you.', 'emotion': 'neutral'},
   'role': 'bot'}],
 [{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'content': {'dialog': 'Hello, how are you?', 'emotion': 'neutral'},
   'role': 'user'},
  {'content': {'dialog': "Hi! I'm terribly nervous.", 'emotion': 'fear'},
   'role': 'bot'}],
 [{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'conten

### Simulate Bot's Possible Emotion Representation

In [30]:
possible_response_emotion_composition: dict = (
    get_possible_response_emotion_representation(
        generated_candidates, emotion_predictor
    )
)
possible_response_emotion_composition

skipping cudagraphs due to mutated inputs (1 instances). Found from : 
   File "/home/hermeschen/Repo/chat-bot/.venv/lib/python3.12/site-packages/transformers/models/roberta/modeling_roberta.py", line 121, in torch_dynamo_resume_in_forward_at_120
    embeddings += position_embeddings



{'neutral': tensor([0.1555, 0.1213, 0.1199, 0.1200, 0.2407, 0.1211, 0.1214]),
 'fear': tensor([0.2466, 0.1217, 0.1203, 0.1285, 0.1364, 0.1227, 0.1238]),
 'happiness': tensor([0.1555, 0.1213, 0.1199, 0.1200, 0.2407, 0.1211, 0.1214]),
 'surprise': tensor([0.1562, 0.1223, 0.1210, 0.1232, 0.2309, 0.1218, 0.1245])}

In [31]:
simulated_emotion_representation: dict = {
    k: emotion_model.forward(v, bot_emotion_representation)
    for k, v in possible_response_emotion_composition.items()
}
simulated_emotion_representation

skipping cudagraphs due to skipping cudagraphs due to cpu device (primals_5). Found from : 
   File "/home/hermeschen/Repo/chat-bot/src/models/libs/EmotionModel.py", line 40, in forward
    decomposed_representation: Tensor = representation.diag().to(



{'neutral': tensor([0.3390, 0.3033, 0.3030, 0.3020, 0.4225, 0.3033, 0.3041],
        device='cuda:0', grad_fn=<CompiledFunctionBackward>),
 'fear': tensor([0.4302, 0.3038, 0.3034, 0.3104, 0.3182, 0.3047, 0.3065],
        device='cuda:0', grad_fn=<CompiledFunctionBackward>),
 'happiness': tensor([0.3389, 0.3032, 0.3030, 0.3019, 0.4226, 0.3033, 0.3041],
        device='cuda:0', grad_fn=<CompiledFunctionBackward>),
 'surprise': tensor([0.3397, 0.3044, 0.3041, 0.3052, 0.4128, 0.3038, 0.3072],
        device='cuda:0', grad_fn=<CompiledFunctionBackward>)}

### Compute Similarity Score

In [32]:
similarity_score: list = similarity_analyzer(
    list(simulated_emotion_representation.values()), ideal_bot_emotion_representation
).tolist()
similarity_score

[0.5040311217308044,
 0.4850131571292877,
 0.5039858818054199,
 0.5041945576667786]

### Get Best Response Emotion

In [33]:
emotions: list = [
    "neutral",
    "anger",
    "disgust",
    "fear",
    "happiness",
    "sadness",
    "surprise",
]
best_bot_response_emotion: str = emotions[
    max(enumerate(similarity_score), key=itemgetter(1))[0]
]
best_bot_response_emotion

'fear'

In [34]:
chat_buffer.append(
    {"role": "bot", "content": {"emotion": best_bot_response_emotion, "dialog": ""}}
)
chat_buffer

[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
 {'role': 'user',
  'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
 {'role': 'bot', 'content': {'emotion': 'fear', 'dialog': ''}}]

### Response

In [35]:
chat_buffer = response_generator(chat_buffer, streamer=streamer)[0]
chat_buffer

   neutral   Hello, how are you?     fear    Hi! I'm scared! 


[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
 {'role': 'user',
  'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
 {'role': 'bot', 'content': {'emotion': 'fear', 'dialog': "Hi! I'm scared!"}}]

In [36]:
if len(chat_buffer) > 11:
    chat_buffer.pop(1)
chat_buffer

[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
 {'role': 'user',
  'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
 {'role': 'bot', 'content': {'emotion': 'fear', 'dialog': "Hi! I'm scared!"}}]